In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-19 15:36:47.564716: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 15:36:48.105759: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [4]:
model = SAT1Topological(8, 5, len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

2023-09-19 15:36:54.747156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 15:36:54.773151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 15:36:54.773224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 15:36:54.775023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-19 15:36:54.775083: I tensorflow/compile

In [5]:
# model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#     metrics=["accuracy"],
# )

In [5]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

Epoch 1/20


2023-09-19 15:36:58.974289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-19 15:36:59.608814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-19 15:36:59.611623: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5602873f9070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-19 15:36:59.611651: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-09-19 15:36:59.615385: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-19 15:36:59.712608: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

765/765 [==============================] - 14s 14ms/step - loss: 0.8279 - accuracy: 0.6827 - val_loss: 0.6943 - val_accuracy: 0.7460
Epoch 2/20
765/765 [==============================] - 10s 13ms/step - loss: 0.5776 - accuracy: 0.7853 - val_loss: 0.5481 - val_accuracy: 0.7936
Epoch 3/20
765/765 [==============================] - 11s 14ms/step - loss: 0.4817 - accuracy: 0.8205 - val_loss: 0.5923 - val_accuracy: 0.7874
Epoch 4/20
765/765 [==============================] - 11s 14ms/step - loss: 0.4136 - accuracy: 0.8506 - val_loss: 0.5591 - val_accuracy: 0.8025
Epoch 5/20
765/765 [==============================] - 11s 14ms/step - loss: 0.3594 - accuracy: 0.8694 - val_loss: 0.6696 - val_accuracy: 0.7981
Epoch 6/20
247/247 [==============================] - 3s 10ms/step


(<keras.src.callbacks.History at 0x7f2c1c124990>, None)

In [15]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 147, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 30, 143, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 71, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 69, 128)       24704     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 34, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 32, 256)       98560 